In [2]:
from http.client import ResponseNotReady
from google.cloud import speech_v1 as speech
import os
import json
from google.protobuf.json_format import MessageToJson

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"D:\UPES\Project\Project Auto_Dub\Local_to_GCP\upheld-beach-360516-95be0c166639.json"


In [3]:
def speech_to_text(config, audio):
    client = speech.SpeechClient()
    response = client.recognize(config=config, audio=audio)
    # print(response)
    return response

In [4]:
def print_sentences(response):
    for result in response.results:
        best_alternative = result.alternatives[0]
        transcript = best_alternative.transcript
        confidence = best_alternative.confidence
        print("-" * 80)
        print(f"Transcript: {transcript}")
        print(f"Confidence: {confidence:.0%}")
        print_word_offsets(best_alternative)

In [5]:
def print_word_offsets(alternative):
    for word in alternative.words:
        confidence = word.confidence
        start_s = word.start_time.total_seconds()
        end_s = word.end_time.total_seconds()
        word = word.word
        print(f"{start_s:>7.3f} | {end_s:>7.3f} | {word} |{confidence:.0%}")
        with open('transcribe.txt', 'a') as the_file:
            the_file.write(f"{start_s:>7.3f} | {end_s:>7.3f} | {word} |{confidence:.0%}\n")

In [6]:
config = dict(
    language_code="en-US",
    enable_automatic_punctuation=True,
    enable_word_time_offsets=True,
    audio_channel_count = 2,
    enable_word_confidence=True)

audio = dict(uri="gs://flac_file111/how-to-speak-so-that-people-want-to-listen-julian-treasure_RRYwKciD.flac")

In [7]:
response = speech_to_text(config, audio)
print(response)

results {
  alternatives {
    transcript: "The human voice. It\'s the instrument. We will play."
    confidence: 0.97041595
    words {
      start_time {
        seconds: 13
        nanos: 700000000
      }
      end_time {
        seconds: 14
        nanos: 100000000
      }
      word: "The"
      confidence: 0.987629056
    }
    words {
      start_time {
        seconds: 14
        nanos: 100000000
      }
      end_time {
        seconds: 14
        nanos: 400000000
      }
      word: "human"
      confidence: 0.987629056
    }
    words {
      start_time {
        seconds: 14
        nanos: 400000000
      }
      end_time {
        seconds: 14
        nanos: 500000000
      }
      word: "voice."
      confidence: 0.987629056
    }
    words {
      start_time {
        seconds: 14
        nanos: 500000000
      }
      end_time {
        seconds: 16
      }
      word: "It\'s"
      confidence: 0.982520401
    }
    words {
      start_time {
        seconds: 16
      }
  

In [8]:
print(type(response))

<class 'google.cloud.speech_v1.types.cloud_speech.RecognizeResponse'>


In [9]:
x = str(response)
text_file = open("response.json", "w")
text_file.write(x)
text_file.close()

In [10]:
# for i in response.results:
#     for word in i.alternatives:
#         all_words = []
#         for j in word.words:            
#             start_time = j.start_time
#             end_time = j.end_time
#             word = j.word
#             confidence = j.confidence
#             row_to_add = [start_time, end_time, word, confidence]
#             all_words.append(row_to_add)
            
            
# print(all_words)
            

In [11]:
# import pandas as pd                                          #csv
# headers = ['Start Time','End Time', "Word", 'Confidence']
# words_df = pd.DataFrame(data = all_words,header = headers)

In [12]:
# with open(r'all_words.csv', 'w') as fp:
    
#     str_to_write='start_time, end_time, word, confidence'
#     fp.write(str_to_write + '\n')
    
#     for row in all_words:
#         row_str = []
#         for item in row:
#             str_item = str(item)
#             row_str.append(str_item)
            
#         str_to_write = ','.join(row_str)
#         # write each item on a new line
#         fp.write(str_to_write + '\n')
#     print('Done')

In [13]:
# for i in response.results:
#     for word in i.alternatives:
#         all_words = []
#         for j in word.words:            
#             word = j.word
#             all_words.append(word)

# print(all_words)      


In [14]:
text = ""

for i in response.results:
    for j in i.alternatives:
        text+=j.transcript 

print(text)
text_500 = text[0:499]
print(text_500)
print(type(transcript))
        

The human voice. It's the instrument. We will play. It's the most powerful sound in the world. Probably, it's the only one that can start a war or say. I love you. And yet many people have the experience that when they speak people, don't listen to them. Why is that? How can we speak to make change in the world? But I like to suggest that we need to move away from. I've I've assembled for your pleasure, his seven deadly sins of speaking. I'm not pretending, this is an exhaustive list but these seven I think I'm pretty large. Haven't said we can all fit into first. Got it, if somebody who's not a nice habit and we know perfectly well the person goes to paying 5 minutes later will be gossiping
The human voice. It's the instrument. We will play. It's the most powerful sound in the world. Probably, it's the only one that can start a war or say. I love you. And yet many people have the experience that when they speak people, don't listen to them. Why is that? How can we speak to make change